In [1]:
#########
# Import
#########
import re
import regex
from collections import Counter
import pandas as pd
import numpy as np
import emoji
import nltk

In [2]:
# import the csv file with all the comments and post togheter
comDB = pd.read_csv(r"database/com_liwc.csv", sep='\t', engine='python', encoding='utf-8')

# import the csv file with JUST the politicians post
postDB = pd.read_csv(r"database/postDB.csv", engine='python')

In [3]:
# create the Data Frame
#df = pd.DataFrame(data=comDB)
df_post = pd.DataFrame(data=postDB)

# add a new colum with sequence numbers
#df['Count']=1
df_post['Count']=1

# print all the DF
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 5)

In [4]:
postDB.head()

,Unnamed: 0,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP,Count,target1_2
0,0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,destra,s,Se i musulmani pensano di portare la guerra sa...,270,80,57,33,1729,0,problematico,Rifugiati Musulmani,religioni europa,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,C'è poco da dire questa è gente che la guerra ...,1,FRATELLIDITALIA.PUGLIA,Se i musulmani pensano di portare la guerra sa...,problematico,probl-hate,incivile,negativo,Rifugiati Musulmani,NaN,Other,Other,4,2.00,50.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0,0.00,0.00,0,0.0,0.0,0.00,0.00,0.00,0.00,125.00,100.00,0.00,0.0,0.00,0.00,25.00,0.0,0.0,0.00,0.0,0.0,6,6.00,33.33,33.33,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,33.33,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0,0.00,0.00,0,0.00,0.0,0.00,0.0,0.00,0.00,16.67,16.67,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,1,NaN
1,15,53320,FB,911961728894076_2169530686470501,2019-04-17T12:50:26Z,IGOR GELARDA,M,EFDD,LEGA,Lega,governo,destra,i,Un mio particolare ringraziamento va ai Magist...,69,33,14,5,6565,0,positivo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Ca

## Posts Text cleaning

In [5]:
# comments = df[['c_text']].sample(n=1000, random_state=1).copy() # work with a sample
##make a copy of the original df
posts = df_post[['p_text']].copy()
posts.rename(columns={'p_text':'text'}, inplace=True)

In [6]:
###############################
# Set seed for reproducibility
###############################

import random
random.seed(0)
np.random.seed(0)

In [7]:
for i in list(np.random.choice(list(posts.index), 5)):
    print(f'Post {i}')
    print(posts.loc[i]['text'], '\n')

Post 2732
Ho capito perché tutti i giorni parlano di #fascismo: hanno bisogno di vendere giornali, libri, riviste. Sono a corto di idee e puntano sull'usato sicuro. Dovrebbero chiedersi perché tanta attenzione su un periodo storico di pochi anni nonostante più di 70 anni di indottrinamento a reti unificate #ScriviMussolini 

Post 9845
Ora in diretta da FAVARA a #lariachetira 

Post 3264
Lui e’ deputato e vicemistro di un governo che ha approvato un piano energia e clima che non rispetta accordi #Onu di #Parigi ,che ha consentito che fanghi di depurazione contenenti diossine possano essere sparsi sui suoli agricoli. #Ipocrisia al governo del paese #sapevatelo https://t.co/xdR9ahi54G 

Post 4859
Ho un brutto carattere (cit) https://t.co/2xPObfzbPz 

Post 9225
Noi andiamo avanti. Siete d'accordo?

https://t.co/AIhLqipcvM 



In [8]:
###Text pre-processing
print('df_post shape before dropping row: \t', df_post.shape)
df_post = df_post[df_post['p_text'].notna()]
print('df_post shape after dropping row: \t', df_post.shape)
print('Number of Nan in comments text: ', df_post['p_text'].isna().sum())

df_post shape before dropping row: 	 (10103, 246)
df_post shape after dropping row: 	 (10102, 246)
Number of Nan in comments text:  0


In [9]:
###make a copy of the original data frame
posts["text_clean"] = posts["text"].copy()

In [10]:
###make the text lower case
def lower_casing(text):
    return(text.lower())

In [11]:
def remove_spaces(text):
    return ' '.join([token for token in text.split()])

In [12]:
posts["text_clean"] = posts["text"].dropna(how = 'all')

In [13]:
posts["text_clean"] = posts["text_clean"].dropna().apply(lambda text: lower_casing(text))

In [14]:
posts["text_clean"] = posts["text_clean"].dropna().apply(lambda text: remove_spaces(text))

In [15]:
posts

,text,text_clean
0,Se i musulmani pensano di portare la guerra sa...,se i musulmani pensano di portare la guerra sa...
1,Un mio particolare ringraziamento va ai Magist...,un mio particolare ringraziamento va ai magist...
...,...,...
10101,🔴Quasi il 60% degli italiani è favorevole alla...,🔴quasi il 60% degli italiani è favorevole alla...
10102,Il mio intervento ieri sera nella trasmissione...,il mio intervento ieri sera nella trasmissione...


In [16]:
##remove patterns
import re
import regex
from collections import Counter

def remove_patterns(text, patterns):
    for pattern in patterns:
        try: r = re.findall(pattern, text)
        except: r = regex.findall(pattern, text)
        for i in r:
            try: text = re.sub(re.escape(i), ' ', text)
            except: text = regex.sub(regex.escape(i), ' ', text)
    return text

def pattern_freq(docs, pattern):
    p_freq = Counter()
    for text in docs:
        p_found= re.findall(pattern, text)
        for p in p_found:
            p_freq[p] += 1
    return p_freq

In [17]:
%%capture
!pip install emoji
import emoji

In [18]:
PATTERNS = {'urls': re.compile(r'https?://\S+|www\.\S+'),
            'users': re.compile(r'@[\w]*'),
            #'hashtags': re.compile(r'#[\w]*'),
            'emojis': emoji.get_emoji_regexp(),
            'laughs': re.compile(r'\b(?:(?:hah+|ah+|hah+a|ah+a|lo+l) ?)+\b'),
            'blabla': re.compile(r'\b(?:(?:bla+) ?)+\b'),
            'dates': re.compile(r'\b(?:(?:[0-9]+[:\/,\.])+[0-9]+)+\b'),
            'digits': r'#\S+(*SKIP)(*FAIL)|\b\d+\s|\s\d+\s|\s\d+|\b\d+\W|\W\d+\W|\W\d+$' 
            }

In [19]:
###remove the patterns
posts["text_clean"] = posts["text_clean"].dropna().apply(lambda text: remove_patterns(text, PATTERNS.values()))

In [20]:
posts

,text,text_clean
0,Se i musulmani pensano di portare la guerra sa...,se i musulmani pensano di portare la guerra sa...
1,Un mio particolare ringraziamento va ai Magist...,un mio particolare ringraziamento va ai magist...
...,...,...
10101,🔴Quasi il 60% degli italiani è favorevole alla...,quasi il % degli italiani è favorevole alla c...
10102,Il mio intervento ieri sera nella trasmissione...,il mio intervento ieri sera nella trasmissione...


In [21]:
##Unicode replacer

from utils.unicode_replacer import *

In [22]:
posts['text_clean'] = posts['text_clean'].dropna().apply(lambda text: UnicodeReplacer().replace(text))

In [23]:
##removing panctuation
import string
punct = string.punctuation
print(f"Punctuation symbols: {punct}")

Punctuation symbols: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [24]:
def remove_punctuation(text, keep=[]):
    punctuation = string.punctuation
    for p in keep:
        punctuation = punctuation.replace(p, "")
    return(''.join([char if not char in punctuation else ' ' for char in text]))

In [25]:
posts['text_clean'] = posts['text_clean'].dropna().apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [26]:
posts

,text,text_clean
0,Se i musulmani pensano di portare la guerra sa...,se i musulmani pensano di portare la guerra sa...
1,Un mio particolare ringraziamento va ai Magist...,un mio particolare ringraziamento va ai magist...
...,...,...
10101,🔴Quasi il 60% degli italiani è favorevole alla...,quasi il degli italiani è favorevole alla cast...
10102,Il mio intervento ieri sera nella trasmissione...,il mio intervento ieri sera nella trasmissione...


### NLP with Spacy

In [27]:
from spacy.lang.it.stop_words import STOP_WORDS as it_spacy_stopwords

In [28]:
import nltk
nltk.download('stopwords')
it_nltk_stopwords = nltk.corpus.stopwords.words('italian')

[nltk_data] Downloading package stopwords to /Users/lynxy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
it_stop_words=[
     'a',
     'adesso',
     'ai',
     'al',
     'alla',
     'allo',
     'allora',
     'almeno',
     'altra',
     'altre',
     'altri',
     'altro',
     'anche',
     'ancora',
    'anno',
     'appena',
     'appieno',
     'avanti',
     'aver',
     'avere',
     'aveva',
     'avevano',
     'basta',
     'bastare',
     'bisogna',
     'bisognare',
     'bo',
     'boh',
     'capire',
     'capisco',
     'capite',
     'capito',
     'c',
     "c'",
     'ce',
     'certo',
     'che',
     'chi',
     'cio',
     'ciò',
     'cm',
     'cmq',
     'cn',
     'come',
     'comunque',
     'con',
     'cosa',
     'cose',
     'cosi',
     'così',
     'credo',
     'cs',
     'cui',
     'd',
     "d'",
     'da',
     'dà',
     'dá',
     'dapprima',
     'data',
     'date',
     'dato',
     'davvero',
     'de',
     'del',
     'dell',
     'della',
     'dello',
     'dentro',
     'detto',
     'deve',
     'devo',
     'devono',
     'di',
     'diciamo',
     'dico',
     'dicono',
     'dite',
     'dobbiamo',
     'dovere',
     'dovete',
     'dovrebbe',
     'dovrebbero',
     'dovresti',
     'doppio',
     'due',
     'e',
     'è',
     'ecco',
     'eccoli',
     'eccolo',
     'er',
     'essere',
     'fare',
     'finalmente',
     'finche',
     'finchè',
     'fino',
     'forse',
     'fra',
     'giu',
     'giusto',
     'guarda',
     'ha',
     'hai',
     'hanno',
     'ho',
     'i',
     'il',
     'immediatamente',
     'infatti',
     'invece',
     'io',
     'k',
     'ke',
     'l',
     "l'",
     'la',
     'le',
     'lei',
     'letto',
     'lo',
     'loro',
     'lui',
     'lungo',
     'ma',
     'magari',
     'man',
     'mano',
     'me',
     'meglio',
     'mentre',
     'mo',
     'modo',
     'molta',
     'molti',
     'molto',
     'neanche',
     'nei',
     'nella',
     'nemmeno',
     'no',
     'noi',
     'nome',
     'non',
     'nn',
     'nostro',
     'notte',
     'nove',
     'nulla',
     'nuova',
     'nuove',
     'nuovi',
     'nuovo',
     'o',
     'oggi',
     'ogni',
     'ok',
     'okay',
     'okey',
     'oltre',
     'ora',
     'oramai',
     'ormai',
     'otto',
     'pare',
     'pensa',
     'pensi',
     'penso',
     'perche',
     'perchè',
     'perché',
     'pero',
     'però',
     'peró',
     'piu',
     'più',
     'piú',
     'poco',
     'possiamo',
     'possibile',
     'posso',
     'possono',
     'posto',
     'prescindere',
     'prima',
     'primo',
     'proprio',
     'ps',
     'puoi',
     'purtroppo',
     'qua',
     'qualche',
     'qualcosa',
     'qualcuno',
    'qualunque',
    'qualsiasi',
     'quarto',
     'quasi',
     'quattro',
     'quel',
     'quello',
     'quest',
     'questa',
     'queste',
     'questi',
     'questo',
     'qua',
    'quando',
     'qui',
     'quindi',
     'quinto',
     'sa',
     'sara',
     'scusa',
     'scusate',
     'secondo',
     'sei',
     'sembra',
     'sembrava',
    'sempre',
     'senso',
     'senza',
     'sette',
     'sia',
     'siamo',
     'siete',
     'solo',
     'sono',
     'sopra',
     'soprattutto',
     'sotto',
     'speriamo',
     'spero',
     'stanotte',
     'stasera',
     'stati',
     'stata',
     'stato',
     'stessa',
     'stesso',
     'sta',
     'ste',
     'sti',
     'sto',
     'su',
     'subito',
     'sul',
     'sulla',
     'tanta',
     'tante',
     'tanti',
     'tanto',
     'te',
     'terzo',
     'tipa',
     'tipe',
     'tipi',
     'tipo',
     'tra',
     'tramite',
     'tre',
     'triplo',
     'troppa',
     'troppe',
     'troppi',
     'troppo',
     'tutta',
     'tutte',
     'tutti',
     'tutto',
     'u',
     'ultima',
     'ultimo',
     'un',
     'una',
     'uno',
     'va',
     'vabbe',
     'vabbè',
     'vai',
     'vanno',
     'vedere',
     'vedo',
     'vengo',
     'venire',
     'venuta',
     'venute',
     'venuti',
     'venuto',
     'vera',
     'veramente',
     'vere',
     'veri',
     'vero',
     'verso',
     'viene',
     'vista',
     'viste',
     'visti',
     'visto',
     'voi',
     'vogliamo',
     'voglio',
     'vogliono',
     'volta',
     'volte',
     'vorrei',
     'vorreste',
     'vorresti',
     'vorrebbero',
     'vostro',
     'vuoi',
     'vuol',
     'vuole',
     'x',
     'xche',
     'xché',
     'xk',
     'xke',
     'xké']

In [30]:
set_stop=set(it_stop_words)

In [31]:
allstop=set(it_nltk_stopwords)|set_stop

In [32]:
def remove_stop(text):
    return(' '.join([word for word in text.split() if word not in allstop]))

In [33]:
def replace_upbra(text):
    return re.sub('“',' ',text)

In [34]:
def replace_tilde(text):
    return re.sub('’',' ',text)

In [35]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_stop(text))
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: replace_upbra(text))
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: replace_tilde(text))

In [36]:
posts['text_clean'] = posts['text_clean'].dropna().apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [37]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_stop(text))

In [38]:
posts['text_clean'] = posts['text_clean'].dropna().apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [39]:
# multiple vowels? multiple consonants?
import re
def remove_duplicated(text):
    return re.sub("(.)\\1{2,}", "\\1", text)

In [40]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_duplicated(text))

In [41]:
##Remove not significant one-char, two-char, three-char
##
word3=[]
for post in posts['text_clean']:
    a=' '.join([word for word in str(post).split() if len(word)<2])
    for word in a:
        word3.append(word)

In [42]:
one_word_set=set(word3)

In [43]:
def remove_ones(text):
    return(' '.join([word for word in text.split() if word not in one_word_set]))

In [44]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_ones(text))

In [45]:
''' CHECK '''
word3=[]
for post in posts['text_clean']:
    a=' '.join([word for word in str(post).split() if len(word)<2])
    for word in a:
        word3.append(word)

In [46]:
word3## successfully removed

[]

In [47]:
#### 2 char words
word4=[]

for post in posts['text_clean']:
    a=[word for word in str(post).split() if len(word)==2]
    if a:
        if len(a)>1:
            for word in a:
                word4.append(word)
        else:
            b=''.join(a)
            word4.append(b)

In [48]:
two_words_set=set(word4)

In [49]:
# manually select relevant acronyms?
# to keep: pd, ok
two_words_set=set(word4)
two_words_set.remove('pd')
#two_words_set.remove('ok')

In [50]:
def remove_two(text):
    return(' '.join([word for word in text.split() if word not in two_words_set]))

In [51]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_two(text))

In [52]:
'''CHECK'''
word4=[]

for post in posts['text_clean']:
    a=[word for word in str(post).split() if len(word)==2]
    if a:
        if len(a)>1:
            for word in a:
                word4.append(word)
        else:
            b=''.join(a)
            word4.append(b)

In [53]:
###remove the three letter words
word5=[]

for post in posts['text_clean']:
    a=[word for word in str(post).split() if len(word)==3]
    if a:
        if len(a)>1:
            for word in a:
                word5.append(word)
        else:
            b=''.join(a)
   

In [54]:
three_words_set=set(word5)

In [59]:
three_words_set.remove('gap')
three_words_set.remove('mld')
three_words_set.remove('ogm')

In [60]:
def remove_three(text):
    return(' '.join([word for word in text.split() if word not in three_words_set]))

In [61]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_three(text))

In [62]:
###LONG WORDS
# spaces and puctuation double check
posts['text_clean'] = posts['text_clean'].dropna().apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [63]:
posts

,text,text_clean
0,Se i musulmani pensano di portare la guerra sa...,musulmani pensano portare guerra santa casa ar...
1,Un mio particolare ringraziamento va ai Magist...,particolare ringraziamento magistrati palermit...
...,...,...
10101,🔴Quasi il 60% degli italiani è favorevole alla...,italiani favorevole castrazione chimica pedofi...
10102,Il mio intervento ieri sera nella trasmissione...,intervento ieri sera trasmissione dritto roves...


In [64]:
for i in list(np.random.choice(list(posts.index), 10)):
    print(f'Post {i}')
    print(posts.loc[i]['text'])
    print(posts.loc[i]['text_clean'], '\n')
    print()

Post 7891
🇪🇺 Il mio intervento di ieri al #Sconcerto del primo maggio a Jesolo 👇🏻

#europpe2019 #scriviGardini

Fratelli d'Italia Giorgia Meloni
intervento ieri #sconcerto maggio jesolo #europpe2019 #scrivigardini fratelli italia giorgia meloni 


Post 4373
@Utente_Generico @CarloCalenda Certo, e col M5S.
 


Post 5874
#IoVotoDinoGiarrusso #IoVotoM5S #Europee2019
#iovotodinogiarrusso #iovotom5s #europee2019 


Post 6744
Oggi alla Sapienza profumo di riscatto e libertà. L'odio non può governare l'Italia e verrà sconfitto
sapienza profumo riscatto liberta odio governare italia verra sconfitto 


Post 3468
Se il problema fosse di opportunità economica il Presidente della Rai ne discuterebbe negli organi aziendali con la dovuta riservatezza. Il problema è evidentemente politico. https://t.co/3oTjzRa0Dj
problema opportunita economica presidente discuterebbe organi aziendali dovuta riservatezza problema evidentemente politico 


Post 705
La cronaca e il video del mio sopralluogo nel quartier

In [65]:
def sanity_check(text):
    return [char for char in str(text) if char not in 'abcdefghijklmnopqrstuwvzxy# ']

In [66]:
posts.dropna()

,text,text_clean
0,Se i musulmani pensano di portare la guerra sa...,musulmani pensano portare guerra santa casa ar...
1,Un mio particolare ringraziamento va ai Magist...,particolare ringraziamento magistrati palermit...
...,...,...
10101,🔴Quasi il 60% degli italiani è favorevole alla...,italiani favorevole castrazione chimica pedofi...
10102,Il mio intervento ieri sera nella trasmissione...,intervento ieri sera trasmissione dritto roves...


In [67]:
spec_char=[]
for i in posts['text_clean'].index:
    a = sanity_check(posts['text_clean'][i])
    if a:
        if len(a)>1:
            for char in a:
                spec_char.append(char)
        else:
            b=''.join(a)
            spec_char.append(b)

In [68]:
def multiple_replace(dicti, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dicti.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dicti[mo.string[mo.start():mo.end()]], text) 

dicti = {
    'à': "a",
    'á' : "a",
    'â' : "a",
    'è' : 'e',
    'é' :'e',
    'ê' :'e',
    'ì' :'i',
    'í' :'i',
    'î' :'i',
    'ï' :'i',
    'ò' : 'o',
    'ó' : 'o',
    'ô' :'o',
    'ö' : 'o',
    'ø' : 'o',
    'ù' :'u',
    'ú' :'u',
    'ü' : 'u',
    'ď' : 'd',
    'ģ' : 'g',
    'ĺ' : 'l',
    'ľ' : 'l',
    'ł' : 'l',
    'ť' : 't',
    'ź' : 'z',
    '𝐀' : 'a',
    '𝐈' : 'i',
    '𝐏' : 'p',
    '𝐑' : 'r',
    '𝐕' : 'v',
    '𝐚' : 'a',
    '𝐜' : 'c',
    '𝐝' : 'd',
    '𝐞' : 'e',
    '𝐡' : 'h',
    '𝐢' : 'i',
    '𝐦' : 'm',
    '𝐧' : 'n',
    '𝐨' : 'o',
    '𝐩' : 'p',
    '𝐫' : 'r',
    '𝐬' : 's',
    '𝐭' : 't',
    '𝗔':'a',
     '𝗕': 'b',
     '𝗖': 'c',
     '𝗗': 'd',
     '𝗘': 'e',
     '𝗙': 'f',
     '𝗚': 'g',
     '𝗜': 'i',
     '𝗠': 'm',
     '𝗡': 'n',
     '𝗢': 'o',
     '𝗣': 'p',
     '𝗤': 'q',
     '𝗥': 'r',
     '𝗦': 's',
     '𝗧': 't',
     '𝗨': 'u',
     '𝗩': 'v',
     '𝗭': 'z',
     '𝙂': 'g',
     '𝙖': 'a',
     '𝙙': 'd',
     '𝙞': 'i',
     '𝙣': 'n',
     '𝙤': 'o',
     '𝙧': 'r',
     '𝙨': 's'
  } 




In [69]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: multiple_replace(dicti,text))

In [70]:
spec_char=[]
for i in posts['text_clean'].index:
    a=sanity_check(posts['text_clean'][i])
    if a:
        if len(a)>1:
            for char in a:
                spec_char.append(char)
        else:
            b=''.join(a)
            spec_char.append(b)
set_spec_char=set(spec_char)

In [71]:
def remove_spec_char(text):
    return(''.join([char for char in str(text) if char not in set_spec_char]))

In [72]:
posts['text_clean']=posts['text_clean'].dropna().apply(lambda text: remove_spec_char(text))

In [73]:
''' CHECK FOR SPECIAL CHAR'''
spec_char=[]
for i in posts['text_clean'].index:
    a=sanity_check(posts['text_clean'][i])
    if a:
        if len(a)>1:
            for char in a:
                spec_char.append(char)
        else:
            b=''.join(a)
            spec_char.append(b)
set_spec_char=set(spec_char)

In [74]:
set(spec_char)

set()

In [75]:
final=posts[posts['text_clean']!='']['text_clean']
final.shape

(10044,)

In [76]:
final.to_csv('Posts_cleaned.csv')

## Lematization

In [77]:
import pandas as pd
import numpy as np
import os

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

import pickle
import networkx as nx
import itertools
from itertools import combinations

import statsmodels
import statsmodels.api as sm
import scipy.stats as stats

import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to /Users/lynxy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [78]:
import it_core_news_lg
nlp = it_core_news_lg.load()

In [79]:
from spacy.tokenizer import _get_regex_pattern

# get default pattern for tokens that don't get split
re_token_match = _get_regex_pattern(nlp.Defaults.token_match)
# add your patterns (here: hashtags and in-word hyphens)
re_token_match = f"({re_token_match}|#\w+|\w+-\w+)"


In [80]:

import re
nlp.tokenizer.token_match = re.compile(re_token_match).match

In [81]:
final=pd.read_csv('Posts_cleaned.csv',index_col=0)

In [82]:
final

,text_clean
0,musulmani pensano portare guerra santa casa ar...
1,particolare ringraziamento magistrati palermit...
...,...
10101,italiani favorevole castrazione chimica pedofi...
10102,intervento ieri sera trasmissione dritto roves...


In [83]:
text_nlp = final["text_clean"].dropna().apply(lambda text: nlp(text))
final['text_nlp'] = text_nlp

In [84]:
tokens = pd.Series(final['text_nlp'])

In [85]:
## save all the tokens
tokens.to_csv('posts_all_tokens.csv')

In [86]:
print(f"{'Token':<20}\t{'Lemma':<20}\t{'POS':<20}\t{'is-stop':<8}\t{'is-punct':<8}")
for token in final['text_nlp'].loc[0]:
    print(f"{token.text:<20}\t{token.lemma_:<20}\t{token.pos_:<20}\t{token.is_punct:^8}")

Token               	Lemma               	POS                 	is-stop 	is-punct
musulmani           	musulmano           	NOUN                	   0    
pensano             	pensare             	AUX                 	   0    
portare             	portare             	AUX                 	   0    
guerra              	guerra              	NOUN                	   0    
santa               	santo               	ADJ                 	   0    
casa                	casa                	NOUN                	   0    
arrivato            	arrivare            	VERB                	   0    
momento             	momento             	NOUN                	   0    
prendere            	prendere            	AUX                 	   0    
provvedimenti       	provvedimento       	NOUN                	   0    
drastici            	drastico            	ADJ                 	   0    
resta               	restare             	VERB                	   0    
bloccare            	bloccare            	AUX          

In [92]:
final.dropna(inplace=True) ###drop na values resulting from cleaning

In [93]:
# cycle through the tokens to extract the lemmas

# initialize an empty row for the lemma in the dataframe
final['lemmas']=0
for i in final.index:
    l=''
#     print(i)
    for token in final['text_nlp'][i]:
        l=l+' '+token.lemma_
    final['lemmas'][i]=l[1:]

<ipython-input-93-9df99a8d9e95>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['lemmas'][i]=l[1:]
/Users/lynxy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [95]:
lemmas = []
for i in final.index:
#     print(type(i))
    for token in final['text_nlp'][i]:
        lemmas.append(token.lemma_)

In [96]:
lemmas_final=pd.DataFrame(final['lemmas'])

In [97]:
lemmas_final.rename(columns={'lemmas':'text_nlp'},inplace=True)

In [98]:
lemmas_final

,text_nlp
0,musulmano pensare portare guerra santo casa ar...
1,particolare ringraziamento magistrato palermit...
...,...
10101,italiano favorevole castrazione chimico pedofi...
10102,intervento ieri sera trasmissione dritto roves...


In [99]:
#save all lemmas as id_lemmas
lemmas_final.to_csv('id_lemmas_posts.csv')

In [100]:
all_lemmas=pd.Series(lemmas)
all_lemmas

0         musulmano
1           pensare
            ...    
191089         poco
191090    accordare
Length: 191091, dtype: object

In [101]:
#save all lemmas
all_lemmas.to_csv('Posts_lemmas_list.csv')

In [102]:
len(all_lemmas.unique())

17582

In [103]:
lemmas_unique=pd.Series(all_lemmas.unique())

In [104]:
#save all unique lemmas
lemmas_unique.to_csv('Posts_lemmas_unique.csv')